In [1]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
from collections import Counter
from string import punctuation
from razdel import tokenize

In [3]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

In [5]:
! pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout, SimpleRNN

import keras
from keras import regularizers
from keras.backend import clear_session
from keras.models import Sequential
from keras import layers

Загрузка и подготовка данных

In [8]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
positive

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive
...,...,...
114906,"Спала в родительском доме, на своей кровати......",positive
114907,RT @jebesilofyt: Эх... Мы немного решили сокра...,positive
114908,"Что происходит со мной, когда в эфире #proacti...",positive
114909,"""Любимая,я подарю тебе эту звезду..."" Имя како...",positive


In [9]:
negative = pd.read_csv('negative.csv', sep = ';', usecols = [3], names = ['text'])
negative['label'] = ['negative'] * len(negative)
negative

,text,label
0,на работе был полный пиддес :| и так каждое за...,negative
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",negative
2,@elina_4post как говорят обещаного три года жд...,negative
3,"Желаю хорошего полёта и удачной посадки,я буду...",negative
4,"Обновил за каким-то лешим surf, теперь не рабо...",negative
...,...,...
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative


In [10]:
df = positive.append(negative)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


**Задание 1**
на токенах с высокой частотой

на токенах со средней частотой

на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.
```



In [11]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

Определение частоты токенов

In [12]:
corpus = [token.text for text in df.text for token in tokenize(text) if token.text not in punctuation]
print(len(corpus))
corpus[:10]

3135394


['first_timee', 'хоть', 'я', 'и', 'школота', 'но', 'поверь', 'у', 'нас', 'то']

In [13]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('не', 69603),
 ('и', 55347),
 ('я', 53243),
 ('в', 53038),
 ('RT', 38080),
 ('на', 35812),
 ('t', 33561),
 ('http', 33072),
 ('co', 33041),
 (':(', 32776)]

In [14]:
vec = CountVectorizer(ngram_range=(1, 1), min_df=1000)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.62      0.63      0.63     27621
    positive       0.65      0.64      0.64     29088

    accuracy                           0.63     56709
   macro avg       0.63      0.63      0.63     56709
weighted avg       0.63      0.63      0.63     56709



In [15]:
vec = CountVectorizer(ngram_range=(1, 1), min_df=100, max_df=1000)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.56      0.70      0.62     22523
    positive       0.76      0.64      0.70     34186

    accuracy                           0.66     56709
   macro avg       0.66      0.67      0.66     56709
weighted avg       0.68      0.66      0.67     56709



In [16]:
vec = CountVectorizer(ngram_range=(1, 1), max_df=100)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.73      0.67      0.70     30831
    positive       0.64      0.71      0.67     25878

    accuracy                           0.69     56709
   macro avg       0.69      0.69      0.69     56709
weighted avg       0.69      0.69      0.69     56709



Токены со средней частотой лучше в определении позитивных сообщений, а с меньшей частотой (до 100 в корпусе) - негативных.

**CountVectorizer**

сравнить count/tf-idf/hashing векторайзеры

In [24]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.76      0.76      0.76     27999
    positive       0.77      0.76      0.76     28710

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



**TfidfVectorizer**

In [25]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.73      0.77      0.75     26840
    positive       0.78      0.75      0.76     29869

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



**HashingVectorizer**

In [26]:
vec = HashingVectorizer(n_features=4)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.39      0.52      0.44     21066
    positive       0.64      0.52      0.58     35643

    accuracy                           0.52     56709
   macro avg       0.52      0.52      0.51     56709
weighted avg       0.55      0.52      0.53     56709

